In [ ]:
# THIS IS RUN IN COLAB!
!pip install datasets evaluate transformers[torch];
!pip install accelerate -U;
!pip install --upgrade transformers;
!pip install wandb;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [3]:
# remove warning from trainer
import wandb
wandb.init(mode="disabled")

In [2]:
# load data from drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pickle
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, Trainer, EarlyStoppingCallback
import numpy as np
import evaluate

In [8]:
# load the file that contains the data for baseline model
with open("/content/drive/My Drive/icpp/for_classification.pickle", "rb") as f:
  dataset = pickle.load(f)


In [9]:
pd.Series(dataset["train"]["label"]).value_counts()

,count
1,366
0,114


In [11]:
def load_model(model_name):
  model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    ignore_mismatched_sizes=True,
    id2label={1: 'relevant', 0: 'not_relevant'},
    label2id={'relevant': 1, 'not_relevant': 0}
  )
  model.to("cuda")
  return model

In [12]:
def tokenizer_function(data, tokenizer):
    return tokenizer(data["text"], truncation=True)

def compute_metrics(test_pred, metric):
    predictions, labels = test_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model_name = "distilbert-base-uncased"

model = load_model(model_name)

#
tokenizer = AutoTokenizer.from_pretrained(model_name)

# # required for trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# load tokenizer
tokenized_dataset = dataset.map(lambda x: tokenizer_function(x, tokenizer), batched=True)

# metric: f1
f1 = evaluate.load("f1")

# training arguments
training_args = TrainingArguments(
    output_dir="../model/",
    logging_steps=32,
    learning_rate=0.05,
    #warmup_steps = 100,
    eval_strategy="steps",
    eval_steps = 50,
    save_strategy="steps",
    save_steps=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    push_to_hub=False,
    report_to=None,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
    optim="adamw_torch"
)

# start trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=lambda x: compute_metrics(x, metric=f1),
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

/tmp/ipython-input-13-3774069875.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,F1
50,62.300400,0.584610,0.867925
100,0.548900,0.545466,0.867925
150,0.540300,0.543332,0.867925
200,0.547400,0.543284,0.867925
250,0.547200,0.543277,0.867925
300,0.543600,0.543296,0.867925


TrainOutput(global_step=300, training_loss=7.214271370569865, metrics={'train_runtime': 96.8306, 'train_samples_per_second': 99.142, 'train_steps_per_second': 3.098, 'total_flos': 1205734820991744.0, 'train_loss': 7.214271370569865, 'epoch': 20.0})

In [16]:
trainer.evaluate(tokenized_dataset["validation"])

{'eval_loss': 0.5432774424552917,
 'eval_f1': 0.8679245283018868,
 'eval_runtime': 0.2286,
 'eval_samples_per_second': 262.431,
 'eval_steps_per_second': 8.748,
 'epoch': 20.0}